In [1]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from matplotlib.pyplot import imshow
%matplotlib inline
from keras.layers.merge import concatenate
from keras.regularizers import l2
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD
import sys
import keras.backend as K
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
def inception(X_input, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
    conv_1x1 = Conv2D(filters=filters_1x1, kernel_size=(1, 1), padding='same', activation='relu')(X_input)

    conv_3x3_reduce = Conv2D(filters=filters_3x3_reduce, kernel_size=(1, 1), padding='same', activation='relu')(X_input)

    conv_3x3 = Conv2D(filters=filters_3x3, kernel_size=(3, 3), padding='same', activation='relu')(conv_3x3_reduce)

    conv_5x5_reduce  = Conv2D(filters=filters_5x5_reduce, kernel_size=(1, 1), padding='same', activation='relu')(X_input)

    conv_5x5 = Conv2D(filters=filters_5x5, kernel_size=(5, 5), padding='same', activation='relu')(conv_5x5_reduce)

    maxpool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(X_input)

    maxpool_proj = Conv2D(filters=filters_pool_proj, kernel_size=(1, 1), strides=(1, 1), padding='same', activation='relu')(maxpool)

    inception_output = concatenate([conv_1x1, conv_3x3, conv_5x5, maxpool_proj], axis=3)  

    return inception_output

In [14]:
def Googlenet(input_shape):
    X_input = Input(shape=input_shape)

    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')(X_input)

    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(X)
    
    X = BatchNormalization(axis=3)(X)

    X = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu')(X)

    X = Conv2D(filters=192, kernel_size=(3, 3), padding='same', activation='relu')(X)
    
    X = BatchNormalization(axis=3)(X)

    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(X)

    X = inception(X, 64, 96, 128, 16, 32, 32)

    X = inception(X, 128, 128, 192, 32, 96, 64)

    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(X)

    X = inception(X, 192, 96, 208, 16, 48, 64)
    
    aux_out_1 = AveragePooling2D((5, 5), strides=(3, 3))(X)
    aux_out_1 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), padding = 'same', activation='relu')(aux_out_1)
    aux_out_1 = Flatten()(aux_out_1)
    aux_out_1 = Dense(1024, activation='relu')(aux_out_1)
    aux_out_1 = Dropout(0.7)(aux_out_1)
    aux_out_1 = Dense(2, activation='softmax')(aux_out_1)

    X = inception(X, 160, 112, 224, 24, 64, 64)

    X = inception(X, 128, 128, 256, 24, 64, 64)

    X = inception(X, 112, 144, 288, 32, 64, 64)
    
    aux_out_2 = AveragePooling2D((5, 5), strides=(3, 3))(X)
    aux_out_2 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), padding = 'same', activation='relu')(aux_out_2)
    aux_out_2 = Flatten()(aux_out_2)
    aux_out_2 = Dense(1024, activation='relu')(aux_out_2)
    aux_out_2 = Dropout(0.7)(aux_out_2)
    aux_out_2 = Dense(2, activation='softmax')(aux_out_2)

    X = inception(X, 256, 160, 320, 32, 128, 128)

    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(X)

    X = inception(X, 256, 160, 320, 32, 128, 128)

    X = inception(X, 384, 192, 384, 48, 128, 128)

    X = AveragePooling2D(pool_size=(7, 7))(X)
    
    X = Dropout(0.4)(X)

    X = Dense(1024, activation='relu')(X)
    
    X = Dense(2, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=[X, aux_out_2, aux_out_1], name='Googlenet')
    
    return model


In [15]:
model = Googlenet((224,224,3))

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_163 (Conv2D)             (None, 112, 112, 64) 9472        input_5[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_39 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_163[0][0]                 
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 56, 56, 64)   256         max_pooling2d_39[0][0]           
__________________________________________________________________________________________________
conv2d_164

In [17]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'], loss_weights=[1., 0.3, 0.3])